In [1]:
import numpy as np

In [2]:
def padAmount(image_size, filter_size, stride = 1, mode = 'valid'):
    if(mode == 'same'):
        lower_bound = (image_size-1)*stride - (image_size-filter_size)
        return int((lower_bound + lower_bound%2)/2)
    elif(mode == 'valid'):
        return 0
    else:
        print('invalid input')
        return -1
    

In [3]:
from PIL import Image

In [4]:
# padding section:
def pad_image(im1_array, kernel, stride, mode = 'same'):
    imgsize_x = im1_array.shape[0]
    imgsize_y = im1_array.shape[1]
    kernel_size = kernel.shape[0]
    pad_x = padAmount(imgsize_x, kernel_size, stride, mode)
    pad_y = padAmount(imgsize_y, kernel_size, stride, mode)
    print('pad_x = ' + str(pad_x))
    print('pad_y = ' + str(pad_y))
    paddedim1_array = np.pad(im1_array, ((pad_x,),(pad_y,)), 'constant', constant_values=(0,))
    padded_img1 = Image.fromarray(paddedim1_array)
    #Image.fromarray(paddedim1_array).show()
    
    return paddedim1_array

In [5]:
 def convolve_matrix(img_arr, kernel, stride = 1):
    kernel_size = kernel.shape[0]
    slice_x = np.arange(kernel_size)
    slice_y = np.arange(kernel_size)
    for_leni = img_arr.shape[0]-kernel_size+1
    for_lenj = img_arr.shape[1]-kernel_size+1
    output_img = []
    for i in range(0, for_leni, stride):
        row = []
        for j in range(0, for_lenj, stride):
            a = slice_x+i
            b = slice_y+j
            row.append(np.sum(img_arr[np.ix_(a,b)]*kernel))
        output_img.append(row)
    return np.array(output_img)

In [10]:
#edge detection:
edge = np.array([[ -1, -1, -1],
                  [-1,  8, -1],
                  [-1, -1, -1]])

#sharpening:
sharp = np.array([[ 0, -1,  0],
                  [-1,  5, -1],
                  [ 0, -1,  0]])

#normalize box blur:
blur = np.array([[  1,  1,  1],
                  [ 1,  1,  1],
                  [ 1,  1,  1]])
blur = 1/9 *blur

#gaussian blur 5*5:
gauss_blur = np.array([[ 1,  4,    6,  4, 1],
                       [ 4, 16,   24, 16, 4],
                       [ 6, 24,   36, 24, 6],
                       [ 4, 16,   24, 16, 4],
                       [ 1,  4,    6,  4, 1]])
gauss_blur = gauss_blur * 1/256

#unsharp masking 5*5:
unsharp = np.array   ([[ 1,  4,    6,  4, 1],
                       [ 4, 16,   24, 16, 4],
                       [ 6, 24, -476, 24, 6],
                       [ 4, 16,   24, 16, 4],
                       [ 1,  4,    6,  4, 1]])
unsharp = unsharp * -1/256

kernels = {
    1:edge,
    2:sharp,
    3:blur,
    4:gauss_blur,
    5:unsharp
}

kernels_name = {
    1:'edge_detection_filter',
    2:'sharp_filter',
    3:'normalized_blur_filter',
    4:'gauss_blur(5x5)_filter',
    5:'unsharp_masking(5x5)_filter'
}


#embossing filter
#[[-1, -1, 0],
# [-1, 0 , 1],
# [0, 1, 1]]
#smoothing filter
#1/9*[[1, 1, 1],
#     [1, 1 , 1],
#     [1, 1, 1]]
#intensified sharpening
#[[-1, -1, -1],
# [-1, 9, -1],
# [-1, -1, -1]]

In [28]:
img_name = input("choose image from {1, 2, 3}: ")
img1 = Image.open(img_name+'.jpg').convert('L')
im1_array = np.array(img1)
#info = np.iinfo(im1_array.dtype)
#im1_array = im1_array.astype(np.integer)/info.max

option_ker = int(input("what filter do you want to apply? (choose from the options below):\n1:edge,\n2:sharp,\n3:blur,\n4:gauss_blur,\n5:unsharp\n"))
kernel = kernels[option_ker]
stride = int(input("please enter the value of stride: "))

mode = input("please enter the mode of the size of the output image:{valid/same}: ")
while(mode!='valid' and mode!='same'):
    mode = input("::*NOTVALID INPUT*:: please re-enter the mode of the size of the output image:{valid/same}: ")

paddedim1_array = pad_image(im1_array, kernel, stride, mode)
out_img1 = Image.fromarray(convolve_matrix(paddedim1_array, kernel, stride))
out_img1 = out_img1.convert('L')
#out_img1.show()
out_img1.save(img_name+"_out_"+kernels_name[option_ker]+".jpg")

choose image from {1, 2, 3}: 3
what filter do you want to apply? (choose from the options below):
1:edge,
2:sharp,
3:blur,
4:gauss_blur,
5:unsharp
5
please enter the value of stride: 1
please enter the mode of the size of the output image:{valid/same}: same
pad_x = 2
pad_y = 2


In [8]:
img1.show()